# Trabalho 01

In [4]:
from math import sin, sqrt
import matplotlib
import random

In [5]:
class GeneticAlgorithm:
    def __init__(self, size, n_generations, n_childrens, mutation, fitness, interval, for_max=True):
        self.size = size
        self.n_generations = n_generations
        self.n_childrens = n_childrens
        self.mutation = mutation
        self.fitness = fitness
        self.interval = interval
        self.for_max = for_max

        self.population = self.init_population()
        self.childrens = []

    def evaluate(self, x, y):
        return self.fitness(x, y)

    def init_population(self):
        population = []
        for i in range(self.size):
            x = random.randint(self.interval[0], self.interval[1])
            y = random.randint(self.interval[0], self.interval[1])
            fitness = self.evaluate(x, y)
            individual = [x, y, fitness]
            population.append(individual)

        return population

    def select_father(self):
        max = len(self.population) - 1
        pos_candidate1 = random.randint(0, max)
        pos_candidate2 = random.randint(0, max)

        pos_father = 0
        
        if(self.population[pos_candidate1][2] > self.population[pos_candidate2][2]):
            pos_father = pos_candidate1
        else:
            pos_father = pos_candidate2              

        return pos_father

    def intersection(self, pos_father1, pos_father2):
        x_c1 = self.population[pos_father1][0]
        x_c2 = self.population[pos_father2][0]
        y_c1 = self.population[pos_father2][1]
        y_c2 = self.population[pos_father1][1]

        fitness_c1 = self.evaluate(x_c1, y_c1)
        fitness_c2 = self.evaluate(x_c2, y_c2)

        return [x_c1, y_c1, fitness_c1], [x_c2, y_c2, fitness_c2]
    
    def mutate(self, children):
        x = random.randint(0, 100)
        y = random.randint(0, 100)

        if(x <= self.mutation):
            children[0] = random.randint(self.interval[0], self.interval[1])
        
        if(y <= self.mutation):
            children[1] = random.randint(self.interval[0], self.interval[1])

        return children
    
    def discard(self):
        ind = 1
        for_max = not self.for_max

        while ind <= self.n_childrens:
            index = self.tournament_selection(2, for_max)
            self.population.pop(index)
            ind +=1

    def min_discard(self, individuals):
        self.population = sorted(individuals, key=lambda x:x[2], reverse=True)
        self.discard()
            
    def max_discard(self, individuals):
        self.population = sorted(individuals, key=lambda x:x[2]) 
        self.discard()

    def tournament_selection(self, n, for_max=True):
        # Seleciona n índices aleatoriamente
        candidate_index = random.sample(range(len(self.population)), n)
        #print(candidates)

        candidates = [(i, self.population[i]) for i in candidate_index]
        
        if(for_max):
            best_index, _ = max(candidates, key=lambda x: x[1][2])
        else:
            best_index, _ = min(candidates, key=lambda x: x[1][2])

        return best_index
        
    def roulette_selection(self):
        sum_fitness = sum(individual[2] for individual in self.population)

        select = random.uniform(0, sum_fitness)

        current = 0

        for individual in self.population:
            current += individual[2]

            if current >= select:
                return individual

    def generate(self):
        n = 1

        while n <= self.n_childrens/2:
            pos_father1 = self.tournament_selection(2, for_max=False)
            pos_father2 = self.tournament_selection(2, for_max=False)

            children1, children2 = self.intersection(pos_father1, pos_father2)

            children1 = self.mutate(children1)
            children2 = self.mutate(children2)

            self.childrens.append(children1)
            self.childrens.append(children2)

            n += 1

    def check_individual_best(self):
        pos_best = len(self.population) - 1
        
        print(f'TAMANHO POPULAÇÃO {len(self.population)}')
        print(self.population)
        print('O melhor individuo: ')
        print('x = ', self.population[pos_best][0])
        print('y = ', self.population[pos_best][1])
        print('fitness = ', self.population[pos_best][2])
        print(f'média fitness = {self.media_fitness()}\n')

    def media_fitness(self):
        sum_fitness = sum(individual[2] for individual in self.population)

        return sum_fitness / len(self.population)

    def init(self):
        count_generations = 1

        while count_generations <= self.n_generations:
            print(f'Geração {count_generations}º:')
            self.childrens = []
            self.generate()
            self.population = self.population + self.childrens
            
            if(self.for_max):
                self.max_discard(self.population)
            else:
                self.min_discard(self.population)

            self.check_individual_best()
            count_generations += 1
    

## 1ª Versão:

<p>a) Proponha e justifique uma codificação, tamanho da população e fitness. Plote a superfície em 3D da função custo e mostre os pontos de ótimo encontrados na melhor simulação.</p>

In [6]:
fitness_v1 = lambda x1, x2: 837.9658 - calc_xi(x1) - calc_xi(x2) 

def calc_xi(x):
    return (x * sin(sqrt(modulo(x))))

def modulo(x):
    if(x<0):
        return -1 * x
    else:
        return x

In [9]:
algorithm = GeneticAlgorithm(size=100, n_childrens=70, n_generations=10, mutation=1, interval=[-500, 500], fitness=fitness_v1, for_max=False)

algorithm.init()

Geração 1º:
TAMANHO POPULAÇÃO 100
[[-201, -420, 1457.6672810811551], [-400, 345, 1296.9538604942777], [-407, 4, 1229.0645550617091], [-197, -232, 1140.3467116973748], [320, 18, 1116.3429954907215], [-20, -453, 1112.9101925935738], [-23, 308, 1111.7910053547803], [289, -486, 1089.452684896091], [-47, -210, 1060.3895178294117], [314, -145, 1049.2378688187325], [255, -470, 1047.830352853024], [-472, 349, 1020.2160957203066], [293, -494, 1012.1459827296359], [332, -23, 1010.2899566074151], [334, 359, 984.2625118890876], [69, -213, 967.0233224872402], [28, 349, 919.7618259906989], [-115, -213, 918.2954381409033], [-199, -264, 901.116055128992], [85, -239, 879.3878555896299], [152, -250, 847.9234863064157], [-307, 308, 836.705085997851], [-472, 227, 826.0284572610872], [358, -248, 802.5202328556164], [482, -26, 796.223564343906], [-153, -92, 792.7290563234405], [-201, 458, 783.9079725300078], [-8, 63, 777.6486774353442], [-201, -316, 761.1282870477746], [-317, -213, 754.883696173731], [-258,